## 24-1. 프로젝트 : 커스텀 프로젝트 직접 만들기
 한국어 버전 KLUE benchmark 만들기 (e9t/nsmc" )
  model(klue/ber-base)를 활용하여 NSMC(Naver Sentiment Movie Corpus) task를 도전
  
  

In [37]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성

In [44]:
## 허깅페이스 데이터 가져오기

from datasets import load_dataset
huggingface_mrpc_dataset = load_dataset("Blpeng/nsmc")


Using custom data configuration Blpeng___nsmc-55757a98c8abea78
Reusing dataset csv (/aiffel/.cache/huggingface/datasets/csv/Blpeng___nsmc-55757a98c8abea78/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
huggingface_mrpc_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 400000
    })
})

In [15]:
# Train datasets의 각 컬럼에 해당하는 요소들을 몇 가지만 뜯어보며, 이것들이 제대로 짝을 지어있는지 확인해볼까요?

#train = huggingface_mrpc_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [16]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




#### 커스텀 데이터셋 만들기

https://huggingface.co/datasets/Blpeng/nsmc  이 데이터는 사용이 불가한가?


Using custom data configuration Blpeng___nsmc-55757a98c8abea78


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.19M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /aiffel/.cache/huggingface/datasets/csv/Blpeng___nsmc-55757a98c8abea78/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

ds의 MRPC 데이터셋은 앞서 살펴본 Huggingface dataset과는 어떤 차이가 있는지 확인

In [32]:
ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'document', 'label'],
        num_rows: 400000
    })
})

In [33]:
# 샘플을 추출
examples = ds['train'].select(range(5))

# 열 이름을 저장 (데이터셋의 features를 사용)
cols = ds['train'].features.keys()

# 샘플 출력
for example in examples:
    for col in cols:
        print(col, ":", example[col])
    print('\n')

Unnamed: 0 : 0
id : 8112052
document : 어릴때보고 지금다시봐도 재밌어요ㅋㅋ
label : 1


Unnamed: 0 : 1
id : 8132799
document : 디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.
label : 1


Unnamed: 0 : 2
id : 4655635
document : 폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
label : 1


Unnamed: 0 : 3
id : 9251303
document : 와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지
label : 1


Unnamed: 0 : 4
id : 10067386
document : 안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
label : 1




#### 다만 데이터가 tf.Tensor형태로 묶여있다는 차이점.. Huggingface dataset가 이중 딕셔너리 내부에 데이터를 리스트 형태로 담았기에, 이와 같은 방식으로 tf dataset을 재구성

In [24]:
# Tensorflow dataset 구조를 python dict 형식으로 변경
# Dataset이 train, validation, test로 나뉘도록 구성
#train_dataset = tfds.as_dataframe(tf_dataset['train'], tf_dataset_info)
#val_dataset = tfds.as_dataframe(tf_dataset['validation'], tf_dataset_info)
#test_dataset = tfds.as_dataframe(tf_dataset['test'], tf_dataset_info)

# dataframe 데이터를 dict 내부에 list로 변경
#train_dataset = train_dataset.to_dict('list')
#val_dataset = val_dataset.to_dict('list')
#test_dataset = test_dataset.to_dict('list')

# Huggingface dataset
#tf_train_dataset = Dataset.from_dict(train_dataset)
#tf_val_dataset = Dataset.from_dict(val_dataset)
#tf_test_dataset = Dataset.from_dict(test_dataset)

## STEP 2. klue/bert-base model 및 tokenizer 불러오기

Huggingface Auto Classes를 이용하는 방법 -- 이번엔 Huggingface에서 Auto Model과 이에 상응하는 tokenizer을 불러오겠습니다.

In [25]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [27]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

데이터셋을 한번에 토크나이징할때 자주 사용하는 기법은 map입니다.

map을 사용하게 되면 Data dictionary에 있는 모든 데이터들이 빠르게 적용시킬 수 있습니다.

우리는 map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True로 주어야 합니다.

In [28]:
hf_dataset = huggingface_mrpc_dataset.map(transform, batched=True)

# train & validation & test split
hf_train_dataset = hf_dataset['train']
hf_val_dataset = hf_dataset['validation']
hf_test_dataset = hf_dataset['test']

NameError: name 'huggingface_mrpc_dataset' is not defined

In [12]:
def transform_tf(batch):
    sentence1 = [s.decode('utf-8') for s in batch['sentence1']]
    sentence2 = [s.decode('utf-8') for s in batch['sentence2']]
    return huggingface_tokenizer(
        sentence1,
        sentence2,
        truncation=True,
        padding='max_length',
        return_token_type_ids=False,
    )

# 토큰화 및 패딩을 적용
tf_train_dataset = tf_train_dataset.map(transform_tf, batched=True)
tf_val_dataset = tf_val_dataset.map(transform_tf, batched=True)
tf_test_dataset = tf_test_dataset.map(transform_tf, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## 23-5. 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test

In [13]:
## Trainer를 활용한 학습

In [14]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

래에서 생성하게 될 Trainer의 인자로 넘겨주어야 할 것 중에 compute_metrics 메소드가 있습니다.

이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정해 두는 것입니다.

In [15]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Trainer에 model, arguments, train_dataset, eval_dataset, compute_metrics를 넣고 train을 진행

In [16]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()
print("슝~")

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.381125,0.845588,0.894825
2,0.499400,0.477627,0.852941,0.901316
3,0.329100,0.517735,0.860294,0.902896


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
The following columns in the evalua

슝~


In [17]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 1725
  Batch size = 8


{'eval_loss': 0.574142575263977,
 'eval_accuracy': 0.832463768115942,
 'eval_f1': 0.8783157894736843,
 'eval_runtime': 57.9518,
 'eval_samples_per_second': 29.766,
 'eval_steps_per_second': 3.727,
 'epoch': 3.0}

In [20]:
## 메모리 비우기
#메모리를 비워줍니다.
del huggingface_model

방금 학습시킨 모델은 Huggingface dataset으로 학습시켰는데요, 앞에서 우리가 만든 커스텀 데이터셋 (tf_train_dataset, tf_val_dataset, tf_test_dataset)으로도 학습을 진행해보고, 결과를 비교해봅시다.

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

huggingface_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)

torch.cuda.empty_cache()

trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=tf_train_dataset,    # training dataset
    eval_dataset=tf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()
print("슝~")


In [ ]:
# test 데이터셋을 이용해 평가해봅니다.

# [[Your Code]]
trainer.evaluate(tf_test_dataset)

## 회고


- 배운 점 : huggingface 를 써봤다
- 아쉬운 점 : 속도가 느려서... 
- 느낀 점 : 이렇게 쓰면 되는구나
- 어려웠던 점 : 처음에 이해를 못했다. 데이터를 어떤 포맷으로 맞춰야하는지?